In [1]:
# to do
# data loader
# kbinsdiscretizer
# embeddings
# transformer

In [2]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import train_test_split

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [3]:
n_embed = 10
d_model = 64
batch_size = 256

In [4]:
input = pd.read_csv('./data/train.csv', index_col=0)
target = 'FloodProbability'
features = [col for col in input.columns if col != target]

X_train, X_val, y_train, y_val = train_test_split(
    input[features],
    input[target],
    test_size=0.2,
    random_state=42
    )

In [19]:
disc = KBinsDiscretizer(
    n_bins=n_embed,
    encode='ordinal',
    strategy='uniform',
    subsample=None
)

train_disc = disc.fit_transform(X_train)
train_tensor = torch.tensor(train_disc, dtype=torch.int32)
val_disc = disc.transform(X_val)
val_tensor = torch.tensor(val_disc, dtype=torch.int32)

xs = {
    'train':train_tensor,
    'val':val_tensor
}

ys = {
    'train':torch.tensor(y_train.values, dtype=torch.float32),
   
    'val':torch.tensor(y_val.values, dtype=torch.float32)
}

def get_batch(split):
    assert split in ['train', 'val']
    idx = torch.randint(len(xs[split]), (batch_size,))
    x = xs[split][idx]
    y = ys[split][idx]
    x, y = x.to(device), y.to(device)
    return x, y

In [33]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.embed = nn.Embedding(len(features), d_model)
        self.linear = nn.Linear(d_model, 1)

    def forward(self, x, y=None):
        out = self.embed(x).sum(axis=1)
        out = self.linear(out).squeeze()

        if y == None:
            loss = None
        else:
            loss = F.huber_loss(out, y)
        return out, loss

In [34]:
x, y = get_batch('train')
m = Model().to(device)

In [35]:
pred, loss = m(x, y)

In [36]:
pred.shape

torch.Size([256])

In [37]:
loss

tensor(9.1547, device='mps:0', grad_fn=<HuberLossBackward0>)